In [5]:
!pip install torch

In [6]:
import torch
print(torch.__version__)

2.0.0+cpu


In [9]:
!pip install torchvision

In [10]:
import torchvision
print(torchvision.__version__)

0.15.1+cpu


In [11]:
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [13]:
#Define the LeNet model
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.pool2(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

#Define training hyperparameters
batch_size = 64
learning_rate = 0.001
num_epochs = 10

In [16]:
#Download training data & test data from MINST dataset
train_dataset = datasets.MNIST(
root='./data',
train=True,
download=True,
transform=ToTensor(),)

test_dataset = datasets.MNIST(
root='./data',
train=False,
download=True,
transform=ToTensor(),)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [17]:
#Instantiate the LeNet model, loss function, and optimizer
model = LeNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [18]:
# Train the model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print training progress every 100 iterations
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))

Epoch [1/10], Step [100/938], Loss: 0.4484
Epoch [1/10], Step [200/938], Loss: 0.2640
Epoch [1/10], Step [300/938], Loss: 0.1778
Epoch [1/10], Step [400/938], Loss: 0.0554
Epoch [1/10], Step [500/938], Loss: 0.0923
Epoch [1/10], Step [600/938], Loss: 0.1176
Epoch [1/10], Step [700/938], Loss: 0.1328
Epoch [1/10], Step [800/938], Loss: 0.0974
Epoch [1/10], Step [900/938], Loss: 0.2045
Epoch [2/10], Step [100/938], Loss: 0.0758
Epoch [2/10], Step [200/938], Loss: 0.0949
Epoch [2/10], Step [300/938], Loss: 0.0200
Epoch [2/10], Step [400/938], Loss: 0.0389
Epoch [2/10], Step [500/938], Loss: 0.0270
Epoch [2/10], Step [600/938], Loss: 0.0677
Epoch [2/10], Step [700/938], Loss: 0.0197
Epoch [2/10], Step [800/938], Loss: 0.0286
Epoch [2/10], Step [900/938], Loss: 0.0512
Epoch [3/10], Step [100/938], Loss: 0.0316
Epoch [3/10], Step [200/938], Loss: 0.0341
Epoch [3/10], Step [300/938], Loss: 0.1629
Epoch [3/10], Step [400/938], Loss: 0.0260
Epoch [3/10], Step [500/938], Loss: 0.1564
Epoch [3/10

In [19]:
# Test the model on the test set
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the LeNet on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the LeNet on the 10000 test images: 98.94 %
